In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Carregar os dados

data = pd.read_excel("Base_Dados_Qualidade_Vinhos.xlsx")
df = pd.DataFrame(data)

In [3]:
df

,wine variation,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,6.6,0.36,0.29,1.60,0.021,24.0,85.0,0.98965,3.41,0.61,12.4,9
1,0,7.4,0.24,0.36,2.00,0.031,27.0,139.0,0.99055,3.28,0.48,12.5,9
2,0,9.1,0.27,0.45,10.60,0.035,28.0,124.0,0.99700,3.20,0.46,10.4,9
3,0,7.1,0.26,0.49,2.20,0.032,31.0,113.0,0.99030,3.37,0.42,12.9,9
4,0,6.9,0.36,0.34,4.20,0.018,57.0,119.0,0.98980,3.28,0.36,12.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299,0,9.1,0.59,0.38,1.60,0.066,34.0,182.0,0.99680,3.23,0.38,8.5,3
6300,0,6.9,0.39,0.40,4.60,0.022,5.0,19.0,0.99150,3.31,0.37,12.6,3
6301,0,7.1,0.49,0.22,2.00,0.047,146.5,307.5,0.99240,3.24,0.37,11.0,3
6302,0,8.3,0.33,0.42,1.15,0.033,18.0,96.0,0.99110,3.20,0.32,12.4,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wine variation        6304 non-null   int64  
 1   fixed acidity         6304 non-null   float64
 2   volatile acidity      6304 non-null   float64
 3   citric acid           6304 non-null   float64
 4   residual sugar        6304 non-null   float64
 5   chlorides             6304 non-null   float64
 6   free sulfur dioxide   6304 non-null   float64
 7   total sulfur dioxide  6304 non-null   float64
 8   density               6304 non-null   float64
 9   pH                    6304 non-null   float64
 10  sulphates             6304 non-null   float64
 11  alcohol               6304 non-null   float64
 12  quality               6304 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 640.4 KB


In [5]:
# Função para normalizar os dados
def normalize(data):
    min_val = np.nanmin(data)  # Lida com N values
    max_val = np.nanmax(data)
    if min_val == max_val:
        return data  # Retornar dados originais
    scaler = MinMaxScaler()
    return scaler.fit_transform(data.values.reshape(-1, 1)).reshape(-1)

In [6]:
# Chamando a função normalizar

# Normalizar todas as colunas numéricas
for coluna in df.select_dtypes(include=[np.number]).columns:
    df[coluna+'_normalized'] = normalize(df[coluna])

# Exibir os dados originais e normalizados em uma tabela
tabela_resultados = df.filter(like='_normalized')
print(tabela_resultados)

      wine variation_normalized  fixed acidity_normalized  \
0                           0.0                  0.231405   
1                           0.0                  0.297521   
2                           0.0                  0.438017   
3                           0.0                  0.272727   
4                           0.0                  0.256198   
...                         ...                       ...   
6299                        0.0                  0.438017   
6300                        0.0                  0.256198   
6301                        0.0                  0.272727   
6302                        0.0                  0.371901   
6303                        0.0                  0.537190   

      volatile acidity_normalized  citric acid_normalized  \
0                        0.186667                0.174699   
1                        0.106667                0.216867   
2                        0.126667                0.271084   
3                      

In [7]:
# Função para codificar características em binário
def encode_binary_feature(feature_values, num_bits):
    # Normalizar os valores
    normalized_values = normalize(feature_values)
    # Quantizar para valores binários usando num_bits
    quantized_values = (normalized_values * (2 ** num_bits - 1)).astype(int)
    # Converter para representação binária
    binary_representation = ''.join(format(val, f'0{num_bits}b') for val in [quantized_values])
	# Retorna a representação binária
    return binary_representation

In [8]:
# Função para codificar todas as características em uma única string binária
def encode_individual(df_row, num_bits_per_feature):
    # Inicializa uma string vazia para armazenar a representação binária do indivíduo
    binary_representation = ""
	# Loop para iterar sobre as colunas do DataFrame (excluindo a última coluna 'quality')
    for column in df_row.index[:-1]:  # Exclua a coluna 'quality'
        # Obtém os valores da característica para a coluna atual
        feature_values = df_row[column]
        # Codifica os valores da característica em uma representação binária e os adiciona à representação do indivíduo
        binary_representation += encode_binary_feature(feature_values, num_bits_per_feature)
	# Retorna a representação binária do indivíduo
    return binary_representation

In [9]:
# Função para gerar uma população inicial de indivíduos
def generate_initial_population(dataframe, population_size, num_bits_per_feature):
    # Inicializa uma lista vazia para armazenar a população gerada
    population = []
	# Loop para criar cada indivíduo na população
    for _ in range(population_size):
        # Amostra um único registro do DataFrame aleatoriamente e codifica um indivíduo
        individual = encode_individual(dataframe.sample(n=1).iloc[0, :], num_bits_per_feature)
        # Adiciona o indivíduo à população
        population.append(individual)
	# Retorna a população inicial gerada
    return population

In [10]:
# Parametros População e Bits
population_size = 10
num_bits_per_feature = 8

In [173]:
# Chamando a função gerar  população inicial
initial_population = generate_initial_population(df, population_size, num_bits_per_feature)

# Exibe a população inicial
for i, individual in enumerate(initial_population):
    print(f'\nPopulação Inicial\n')
    print(f'Indivíduo {i+1}: {individual}\n')


População Inicial

Indivíduo 1: 00000000101111110100011001101000010110110001100000110010101111010101011100101000111111110111001000110110101110101111010110111111010000000000010111000010100001001110100011100010000000100100011001101110000010101000001110001101110


População Inicial

Indivíduo 2: 000000001111001001001001111001110101100101001100000110110011111011001000101001110101111111101100100101011111111001101010011111111100000000000101000000100111001000111000110000010011000100000100111011011110010101100010100000111110


População Inicial

Indivíduo 3: 00000000110011110010010110101010110110011000000010101010101101010110010011011011001111110011010011101001011010110111010011111111000000000000011100000010001001101000000101100001101001001010101001101010000011101100011010110000001


População Inicial

Indivíduo 4: 000000001111111100001001001011111001011101001110000100010011111011000100000100111110111111101110010000001010110101011000010100111110110000000001011000001000110100101110000000000010

In [12]:
# Função para decodificar a string binária de volta para os valores originais
def decode_individual(binary_string, num_bits_per_feature):
    decoded_values = []

    # Divida a string binária em pedaços correspondentes ao número de bits por característica
    for i in range(0, len(binary_string), num_bits_per_feature):
        feature_binary = binary_string[i:i+num_bits_per_feature]
        decoded_values.append(int(feature_binary, 2) / (2 ** num_bits_per_feature - 1))

    return np.array(decoded_values)

In [13]:
# Chamando a função de decodificação dos indíviduos na população inicial

# Exibe a decodificação dos indivíduos na população inicial
for i, individual in enumerate(initial_population):
    decoded_values = decode_individual(individual, num_bits_per_feature)
    print(f'Decodificação do Indivíduo {i+1}: {decoded_values}')

Decodificação do Indivíduo 1: [0.         0.8745098  0.14509804 0.65882353 0.3372549  0.31372549
 0.73333333 0.45490196 0.6745098  0.3254902  0.99607843 0.9254902
 0.05882353 0.33333333 0.31764706 0.74509804 0.25098039 0.0627451
 0.76862745 0.28627451 0.76078431 0.2627451  0.5372549  0.8
 0.10196078 0.88235294 0.03529412 0.34509804 0.01176471]
Decodificação do Indivíduo 2: [1.         0.50980392 0.72941176 0.20392157 0.30980392 0.94117647
 0.76862745 0.98431373 0.6        0.40392157 0.99215686 0.56078431
 0.30980392 0.28235294 0.52156863 0.98039216 1.         0.36078431
 0.37254902 0.16078431 0.0627451  0.14117647 0.01176471 0.16078431
 0.8        0.31764706 0.22352941 0.15686275]
Decodificação do Indivíduo 3: [0.         0.74901961 0.2745098  0.6745098  0.09803922 0.50196078
 0.70980392 0.91764706 0.76470588 0.23921569 0.97647059 0.62352941
 0.65882353 0.87058824 0.05098039 0.98039216 0.         0.18039216
 0.08627451 0.22745098 0.00784314 0.01568627 0.07058824 0.21176471
 0.17647059 

In [14]:
# Função para avaliar um indivíduo (aptidão/fitness)
def evaluate_individual(binary_string, num_bits_per_feature):
    # Decodifica a string binária para obter os valores das características
    decoded_values = decode_individual(binary_string, num_bits_per_feature)
    # Calcula o valor da aptidão de cada indivíduo que é a média dos valores decodificados
    quality_score = np.mean(decoded_values)
    # Retorna o valor de aptidão
    return quality_score

In [76]:
# Chamando a função de avaliação dos indivíduos
# Avalia os indivíduos na população inicial
for i, individual in enumerate(initial_population):
    fitness_score = evaluate_individual(individual, num_bits_per_feature)
    print(f'Aptidão do Indivíduo {i+1}: {fitness_score}')
    print(f'Aptidão do Relativa do Indivíduo {i+1}: {fitness_score * 100:.2f}%')

Aptidão do Indivíduo 1: 0.4482758620689655
Aptidão do Relativa do Indivíduo 1: 44.83%
Aptidão do Indivíduo 2: 0.49523809523809526
Aptidão do Relativa do Indivíduo 2: 49.52%
Aptidão do Indivíduo 3: 0.39593837535014004
Aptidão do Relativa do Indivíduo 3: 39.59%
Aptidão do Indivíduo 4: 0.3724137931034482
Aptidão do Relativa do Indivíduo 4: 37.24%
Aptidão do Indivíduo 5: 0.49100743745774167
Aptidão do Relativa do Indivíduo 5: 49.10%
Aptidão do Indivíduo 6: 0.5016903313049358
Aptidão do Relativa do Indivíduo 6: 50.17%
Aptidão do Indivíduo 7: 0.521974306964165
Aptidão do Relativa do Indivíduo 7: 52.20%
Aptidão do Indivíduo 8: 0.5018255578093307
Aptidão do Relativa do Indivíduo 8: 50.18%
Aptidão do Indivíduo 9: 0.49249492900608516
Aptidão do Relativa do Indivíduo 9: 49.25%
Aptidão do Indivíduo 10: 0.5246788370520621
Aptidão do Relativa do Indivíduo 10: 52.47%


In [68]:
# Função para avaliar a população
def evaluate_population(population, num_bits_per_feature):
    # Avalia cada indivíduo na população usando a função evaluate_individual
    evaluated_population = [(ind, evaluate_individual(ind, num_bits_per_feature)) for ind in population]
    # Retorna a população avaliada, onde cada elemento é uma tupla (indivíduo, valor da aptidão)
    return evaluated_population

In [78]:
# Aplica a função de avaliação da população
# Chamando a função de Avaliação da população atual (fitness/aptidão)
evaluated_population = evaluate_population(initial_population, num_bits_per_feature)

#Chamando a função para avaliar a aptidão relativa
# Avalia os indivíduos na população inicial
population_fitness = [evaluate_individual(individual, num_bits_per_feature) for individual in initial_population]

for i, individual in enumerate(initial_population):
    fitness_score = evaluate_individual(individual, num_bits_per_feature)
    relative_fitness = evaluate_relative_fitness(fitness_score, population_fitness)
    #print(f'Aptidão Relativa do Indivíduo {i+1}: {relative_fitness:.2f}%')
    
# Imprima os resultados
sorted_population = sorted(evaluated_population, key=lambda x: x[1], reverse=True)
for i, (individual, quality_score) in enumerate(sorted_population):
    print(f"Indivíduo {i+1}: {individual}")
    print(f'Aptidão do Indivíduo {i+1}: {fitness_score}')
    print(f'Aptidão do Relativa do Indivíduo {i+1}: {fitness_score * 100:.2f}%\n')

Indivíduo 1: 111111111001011101101000110010101000100100101001100010101111110100100100110110111111111011001010001010101010011001000010011111011111111110111100001001111011001010001001110100000000010010001001011011011010110000100000000111011
Aptidão do Indivíduo 1: 0.5246788370520621
Aptidão do Relativa do Indivíduo 1: 52.47%

Indivíduo 2: 00000000110011110010100001001101101100011011101000101011100011100111010101010010101011111111010110101100111111110001111011110011111011000000000011100000011110010000100110000000011111001010110110000010111001000110010010100000100100
Aptidão do Indivíduo 2: 0.5246788370520621
Aptidão do Relativa do Indivíduo 2: 52.47%

Indivíduo 3: 1111111110001100010000111101100010011001111101000101011100011100111111000110001111111111011011000001000100110010010101010011111011111111110110100100011011010100100001011000011111000101010001110111011101100001000010110100101100
Aptidão do Indivíduo 3: 0.5246788370520621
Aptidão do Relativa do Indivíduo 3: 52.47%

Indivíduo 4: 000

In [79]:
# Função de Elitismo
def elitism(evaluated_population, elite_fraction=0.1):
    # Ordena a população avaliada pelos valores de aptidão em ordem decrescente
    sorted_population = sorted(evaluated_population, key=lambda x: x[1], reverse=True)
    
    # Calcula o número de indivíduos a serem incluídos na elite
    elite_size = int(len(evaluated_population) * elite_fraction)
    
    # Separa os melhores indivíduos na lista elite_population
    elite_population = [individual for individual, fitness in sorted_population[:elite_size]]
    
    # Os demais indivíduos (não incluídos na elite) vão para a lista rest_population
    rest_population = [individual for individual, fitness in sorted_population[elite_size:]]
    
    return elite_population, rest_population

In [91]:
#  função de Elitismo
# Avalia os indivíduos na população inicial
evaluated_population  = evaluate_population(initial_population, num_bits_per_feature)

# Aplica elitismo
elite_population, rest_population = elitism(evaluated_population)

# Exibe os indivíduos na elite
print("Elite Population:\n")
for i, individual in enumerate(elite_population):
    print(f'Indivíduo {i+1}: {individual}')

# Exibe os demais indivíduos
print("\nRest Population:\n")
for i, individual in enumerate(rest_population):
    print(f'Indivíduo {i+1}: {individual}\n')

Elite Population:

Indivíduo 1: 111111111001011101101000110010101000100100101001100010101111110100100100110110111111111011001010001010101010011001000010011111011111111110111100001001111011001010001001110100000000010010001001011011011010110000100000000111011

Rest Population:

Indivíduo 1: 00000000110011110010100001001101101100011011101000101011100011100111010101010010101011111111010110101100111111110001111011110011111011000000000011100000011110010000100110000000011111001010110110000010111001000110010010100000100100

Indivíduo 2: 1111111110001100010000111101100010011001111101000101011100011100111111000110001111111111011011000001000100110010010101010011111011111111110110100100011011010100100001011000011111000101010001110111011101100001000010110100101100

Indivíduo 3: 00000000111001011000011001101011011100111110100000111101011110101010011100011000111111110111011101110111110010101000111110111111010000000000100011100010100001101110001011000001000000100100101100010000100100101100010011001100

In [113]:
# Função de Seleção por Roleta
def roulette_selection(evaluated_population):
    # Calcula a soma total das pontuações de qualidade na população avaliada
    total_fitness = sum(fitness for _, fitness in evaluated_population)
    # Calcula as probabilidades de seleção proporcional às pontuações de qualidade
    selection_probs = [fitness / total_fitness for _, fitness in evaluated_population]
    # Realiza a seleção baseada na roleta, escolhendo índices com base nas probabilidades calculadas
    selected_indices = np.random.choice(len(evaluated_population), size=len(evaluated_population), p=selection_probs)
    # Obtém os indivíduos selecionados com base nos índices escolhidos
    selected_individuals = [evaluated_population[i][0] for i in selected_indices]
    # Retorna os indivíduos selecionados
    return selected_individuals

In [114]:
# Chamando a função de seleção por Roleta
# Avalia a aptidão dos indivíduos na rest_population
rest_population_fitness = evaluate_population(rest_population, num_bits_per_feature)

# Aplica a seleção por roleta na rest_population
selected_individuals = roulette_selection(rest_population_fitness)

# Exibe os indivíduos selecionados pela roleta
print("\nOrdem de Seleção dos Indivíduos pela Roleta:\n")
for i, individual in enumerate(selected_individuals):
    print(f'{i+1}ª Indivíduo Selecionado na roleta: {individual}\n')


Ordem de Seleção dos Indivíduos pela Roleta:

1ª Indivíduo Selecionado na roleta: 00000000111001011000011001101011011100111110100000111101011110101010011100011000111111110111011101110111110010101000111110111111010000000000100011100010100001101110001011000001000000100100101100010000100100101100010011001100000

2ª Indivíduo Selecionado na roleta: 000000001101001001100100110010101101111111100001001101100110100111001110101100011111110010111100100111111110111010011110111111010000000000011101100001011001101000000010000001011001001100010101000101111001011110010100010001000

3ª Indivíduo Selecionado na roleta: 00000000110011110010100001001101101100011011101000101011100011100111010101010010101011111111010110101100111111110001111011110011111011000000000011100000011110010000100110000000011111001010110110000010111001000110010010100000100100

4ª Indivíduo Selecionado na roleta: 0000000011001111001010000100110110110001101110100010101110001110011101010101001010101111111101011010110011111111000111101

In [150]:
# Função para selecionar quais invíduos passaram pelo operador de crossover e de mutação
def divide_selected_individuals(selected_individuals, crossover_rate, mutation_rate):
    num_individuals = len(selected_individuals)

    # Calcula o número de indivíduos para crossover e mutação
    num_crossover = max(2, int(crossover_rate * num_individuals // 2) * 2)  # Garante que seja par
    num_mutation = max(1, num_individuals - num_crossover)

    # Seleciona aleatoriamente os índices para crossover e mutação
    crossover_indices = np.random.choice(num_individuals, size=num_crossover, replace=False)
    
    # Filtra os indivíduos para crossover e mutação
    crossover_population = [selected_individuals[i] for i in crossover_indices]
    mutation_population = [selected_individuals[i] for i in range(num_individuals) if i not in crossover_indices]

    return crossover_population, mutation_population

In [162]:
# Parametros de cruzamento e mutação:
crossover_rate = 0.9  # Taxa de crossover entre 0.6 (60 %) e 0.9 (90%)
mutation_rate = 0.01  # Taxa de mutação

In [163]:
# Chamando a função de divisão
# Aplica a divisão
crossover_population, mutation_population = divide_selected_individuals(selected_individuals, crossover_rate, mutation_rate)

# Exibe os resultados
print("\nPais selecionados para Crossover:\n")
for i, individual in enumerate(crossover_population):
    print(f'Pai {i+1}: {individual}\n')

print("\nPai(s) selecionados para Mutação:\n")
for i, individual in enumerate(mutation_population):
    print(f'Pai {i+1}: {individual}')


Pais selecionados para Crossover:

Pai 1: 0000000011011111001001011010100001010110010100001011101101110100101011000101001111111110111011000000111101010101010100011011111001000000000100001100010001001001110000100100001110001001110011000001101011100001000010010101100011

Pai 2: 00000000111001011000011001101011011100111110100000111101011110101010011100011000111111110111011101110111110010101000111110111111010000000000100011100010100001101110001011000001000000100100101100010000100100101100010011001100000

Pai 3: 000000001101001001100100110010101101111111100001001101100110100111001110101100011111110010111100100111111110111010011110111111010000000000011101100001011001101000000010000001011001001100010101000101111001011110010100010001000

Pai 4: 00000000110011110010100001001101101100011011101000101011100011100111010101010010101011111111010110101100111111110001111011110011111011000000000011100000011110010000100110000000011111001010110110000010111001000110010010100000100100

Pai 5: 0000000011101

In [164]:
# Função de Crossover
# operador de crossover de ponto único (single-point crossover)
def crossover(parent1, parent2):
    # Escolhe um ponto de cruzamento aleatório entre 1 e o comprimento da string - 1
    crossover_point = random.randint(1, len(parent1) - 1)

    # Realiza o cruzamento para gerar dois filhos
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    # Retorna os dois filhos resultantes do cruzamento
    return child1, child2

In [165]:
# Função para aplicar crossover na população de crossover
def apply_crossover(crossover_population):
    crossed_children = []

    for i in range(0, len(crossover_population), 2):
        # Aplica o crossover para pares consecutivos na população de crossover
        parent1, parent2 = crossover_population[i], crossover_population[i + 1]
        child1, child2 = crossover(parent1, parent2)

        crossed_children.append(child1)
        crossed_children.append(child2)

    return crossed_children

In [166]:
# Chamando a função de crossover
# Aplica o crossover
crossed_children = apply_crossover(crossover_population)

# Exibe os resultados
print("\nIndivíduos após Crossover:\n")
for i, child in enumerate(crossed_children):
    print(f'Filho {i+1}: {child}\n')


Indivíduos após Crossover:

Filho 1: 00000000110111110010010110101000010101101110100000111101011110101010011100011000111111110111011101110111110010101000111110111111010000000000100011100010100001101110001011000001000000100100101100010000100100101100010011001100000

Filho 2: 0000000011100101100001100110101101110011010100001011101101110100101011000101001111111110111011000000111101010101010100011011111001000000000100001100010001001001110000100100001110001001110011000001101011100001000010010101100011

Filho 3: 00000000110100100110010011001010110111111110000100110110011010011100111010110001111111001011110010011111111011101001111011111101000000000001110110000101100110100000001000000101100100110001010100010110111001000110010010100000100100

Filho 4: 000000001100111100101000010011011011000110111010001010111000111001110101010100101010111111110101101011001111111100011110111100111110110000000000111000000111100100001001100000000111110010101101100000111001011110010100010001000

Filho 5: 0000000011

In [167]:
# Função de mutação
def mutation(individual):
    # Converte o indivíduo em uma lista mutável para realizar a mutação
    mutated_individual = list(individual)
    # Itera sobre cada bit do indivíduo
    for i in range(len(mutated_individual)):
        # Inverte o bit atual (de '0' para '1' ou de '1' para '0')
        mutated_individual[i] = '0' if mutated_individual[i] == '1' else '1'
    mutated_result = ''.join(mutated_individual)
    # Retorna o indivíduo mutado como uma string
    return mutated_result

In [168]:
# Chamando a função de mutação
# Convertendo os indivíduos para strings antes de aplicar a mutação
mutation_population_str = [str(individual) for individual in mutation_population]

# Aplica a mutação na população de mutação
mutated_children = [mutation(individual) for individual in mutation_population_str]

# Exibe os resultados
print("Filho(s) Após Mutação:")
for i, individual in enumerate(mutated_children):
    print(f'Filho(s) {i+1}: {individual}')

Filho(s) Após Mutação:
Filho(s) 1: 11111111000110100111100110010100100011000001011111000010100001010101100011100111000000001000100010001000001101010111000001000000101111111111011100011101011110010001110100111110111111011011010011101111011011010011101100110011111


In [169]:
# Função de consolidação da nova população
def combine_populations(elite_population, crossed_children, mutated_children):
    # Juntando as populações
    combined_population = elite_population + crossed_children + mutated_children
    return combined_population

In [174]:
# Chamando a função de consolidação da nova população
new_population = combine_populations(elite_population, crossed_children, mutated_children)

# Exibindo o resultado
print("Nova população:\n")
for i, individual in enumerate(new_population):
    print(f'Indivíduo {i+1}: {individual}\n')

Nova população:

Indivíduo 1: 111111111001011101101000110010101000100100101001100010101111110100100100110110111111111011001010001010101010011001000010011111011111111110111100001001111011001010001001110100000000010010001001011011011010110000100000000111011

Indivíduo 2: 00000000110111110010010110101000010101101110100000111101011110101010011100011000111111110111011101110111110010101000111110111111010000000000100011100010100001101110001011000001000000100100101100010000100100101100010011001100000

Indivíduo 3: 0000000011100101100001100110101101110011010100001011101101110100101011000101001111111110111011000000111101010101010100011011111001000000000100001100010001001001110000100100001110001001110011000001101011100001000010010101100011

Indivíduo 4: 00000000110100100110010011001010110111111110000100110110011010011100111010110001111111001011110010011111111011101001111011111101000000000001110110000101100110100000001000000101100100110001010100010110111001000110010010100000100100

Indivíduo 5: 00

In [175]:
# Realiviando nova população
# Chamando a função de Avaliação da população atual (fitness/aptidão)
evaluated_population = evaluate_population(new_population, num_bits_per_feature)

#Chamando a função para avaliar a aptidão relativa
# Avalia os indivíduos na população inicial
population_fitness = [evaluate_individual(individual, num_bits_per_feature) for individual in initial_population]

for i, individual in enumerate(initial_population):
    fitness_score = evaluate_individual(individual, num_bits_per_feature)
    relative_fitness = evaluate_relative_fitness(fitness_score, population_fitness)
    #print(f'Aptidão Relativa do Indivíduo {i+1}: {relative_fitness:.2f}%')
    
# Imprima os resultados
sorted_population = sorted(evaluated_population, key=lambda x: x[1], reverse=True)
for i, (individual, quality_score) in enumerate(sorted_population):
    print(f"Indivíduo {i+1}: {individual}")
    print(f'Aptidão do Indivíduo {i+1}: {fitness_score}')
    print(f'Aptidão do Relativa do Indivíduo {i+1}: {fitness_score * 100:.2f}%\n')

Indivíduo 1: 11111111100011000100001111011000100110011111010001010111000111001111110001100011111111110110110000010001001100100101010100111110111111111101101001000110110101001000010110000111110001010100011101110111100011001111110010001000100001
Aptidão do Indivíduo 1: 0.5663865546218487
Aptidão do Relativa do Indivíduo 1: 56.64%

Indivíduo 2: 111111111001011101101000110010101000100100101001100010101111110100100100110110111111111011001010001010101010011001000010011111011111111110111100001001111011001010001001110100000000010010001001011011011010110000100000000111011
Aptidão do Indivíduo 2: 0.5663865546218487
Aptidão do Relativa do Indivíduo 2: 56.64%

Indivíduo 3: 00000000110011110010100001001101101100011011101000101011100011100111010101010010101011111111010110101100111111110001111011110011111011000000000011100000011110010000100110000000011111001010110110000010111001000110010010100000100001
Aptidão do Indivíduo 3: 0.5663865546218487
Aptidão do Relativa do Indivíduo 3: 56.64%

Indivíduo 4: